In [11]:
import pandas as pd
# import sweetviz as sv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
#from pandas_profiling import ProfileReport
import utils

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# 1. Description
The data we have is from the WESAD study and contains labled data (stress/no stress). The data contains data from differente sensors: ACC (accelerometer), BVP (blood volume pulse), EDA (electrodermal activity), TEMP (temperature). In the following, we look at the features created with the FLIRT (https://flirt.readthedocs.io/en/latest/) library (see other script).

Note: there might be other potential features to be calculated on the raw data, for example via tsfresh (https://tsfresh.readthedocs.io/en/latest/index.html) or TSFEL (https://tsfel.readthedocs.io/en/latest/). However, FLIRT was specifically developed with the wrist sensor used in the two dataset used here, so we can reasonably expect it to produce meaningful features based on the available data.

# 2. Data Source

In [3]:
# load data - features calculated with Flirt with
# window_length = 60 and
# window_step_size = 10
df = pd.read_parquet('data-input/flirt-wesad-acc-bvp-eda-temp60-10.parquet')

In [4]:
df.shape

(2765, 222)

In [5]:
df.head(3)

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,temp_l2_n_sign_changes,temp_l2_iqr,temp_l2_iqr_5_95,temp_l2_pct_5,temp_l2_pct_95,temp_l2_entropy,temp_l2_perm_entropy,temp_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:01:00,0.124318,66.395699,-264.71,272.99,537.7,477.38,1.692827e+07,0.023441,2.820353,145.0,...,0,0.04,0.09,33.25,33.34,5.480639,0.802811,0.003120,10,0
1970-01-01 00:01:10,0.843435,58.168574,-264.71,272.99,537.7,3238.79,1.299569e+07,0.112281,4.464214,149.0,...,0,0.02,0.05,33.29,33.34,5.480639,0.795881,0.003081,10,0
1970-01-01 00:01:20,0.111445,53.332683,-264.71,272.99,537.7,427.95,1.092245e+07,-0.007924,6.198772,153.0,...,0,0.02,0.03,33.31,33.34,5.480639,0.795881,0.002733,10,0


In [6]:
# there are no missing values in the dataset
df.isnull().sum().value_counts()

0    222
dtype: int64

In [7]:
df['subject'].value_counts()

10    191
17    191
15    187
11    186
14    186
16    186
13    185
5     185
8     184
6     183
7     183
9     183
4     180
3     178
2     177
Name: subject, dtype: int64

In [8]:
df['label'].value_counts(normalize=True)

0    0.637613
1    0.362387
Name: label, dtype: float64

# 3. Train-test split
We perform the train-test split before we conduct EDA on the train set. Thus, we avoid data leakage from the test set.

In [13]:
df_train, df_test = utils.create_train_test(df, 5, 'subject', 'label')

In [20]:
df_train.shape

(2213, 222)

In [21]:
df_test.shape

(552, 222)

In [19]:
df_train['label'].value_counts(normalize=True)

0    0.6385
1    0.3615
Name: label, dtype: float64

In [18]:
df_test['label'].value_counts(normalize=True)

0    0.634058
1    0.365942
Name: label, dtype: float64

# 4. EDA

## 4.1 Looking into data

In [22]:
# we do not have categorical features, only int (count) and float
df_train.dtypes.value_counts()

float64    204
int32       13
int64        5
dtype: int64

In [23]:
df_train.describe()

C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,temp_l2_n_sign_changes,temp_l2_iqr,temp_l2_iqr_5_95,temp_l2_pct_5,temp_l2_pct_95,temp_l2_entropy,temp_l2_perm_entropy,temp_l2_svd_entropy,subject,label
count,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2.213000e+03,2213.000000,2213.000000,2213.000000,...,2213.0,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000,2213.000000
mean,0.065898,52.491254,-297.012503,267.730547,564.743050,60.119232,1.671780e+07,-0.429217,8.766843,125.206055,...,0.0,0.052746,0.103395,32.893762,32.997157,5.428066,0.730105,0.003381,9.467239,0.361500
std,2.112547,42.297807,252.941324,225.661887,459.450578,2134.374816,3.355718e+07,0.976006,12.518595,31.708195,...,0.0,0.054605,0.088293,1.459188,1.471113,0.271013,0.084469,0.001157,4.260657,0.480544
min,-8.031469,2.985483,-1617.860000,6.350000,14.860000,-12746.100000,5.776164e+03,-8.923326,-1.129112,6.000000,...,0.0,0.000000,0.010000,29.350000,29.430000,2.484907,0.286397,0.001383,3.000000,0.000000
25%,-0.120737,27.294612,-382.280000,115.320000,238.200000,-443.830000,2.697575e+06,-0.739812,2.080863,111.000000,...,0.0,0.020000,0.050000,32.249000,32.291000,5.480638,0.673947,0.003001,6.000000,0.000000
50%,-0.000122,39.260683,-224.110000,212.850000,435.900000,-0.470000,5.849316e+06,-0.378886,4.990000,124.000000,...,0.0,0.040000,0.080000,33.070000,33.150000,5.480639,0.743334,0.003252,10.000000,0.000000
75%,0.124318,64.733161,-115.160000,339.800000,739.430000,451.850000,1.574123e+07,0.054329,10.338357,142.000000,...,0.0,0.060000,0.120000,33.970000,34.090000,5.480639,0.795881,0.003552,14.000000,1.000000
max,90.985016,307.465280,-8.510000,1789.000000,3406.860000,29024.220000,3.630259e+08,4.361864,138.047003,271.000000,...,0.0,0.407500,0.700000,35.610000,35.680000,5.480639,0.973835,0.028596,16.000000,1.000000


In [24]:
# remove rows with only one value for each row
#overall_length = len(df_train)
columns = df_train.columns.tolist()
constant_columns = []

for c in columns:
    unique_in_column = len(df_train[c].unique())
    
    #if unique_in_column/overall_length < 0.1 and c != 'label' and c != 'subject':
    if unique_in_column == 1:
        constant_columns.append(c)

In [25]:
constant_columns

['bvp_BVP_entropy',
 'acc_l2_n_sign_changes',
 'eda_EDA_n_sign_changes',
 'eda_l2_n_sign_changes',
 'temp_TEMP_peaks',
 'temp_TEMP_n_sign_changes',
 'temp_l2_peaks',
 'temp_l2_n_sign_changes']

In [26]:
# remove rows where we cannot calculate sandard deviation of the column

df_desc = df_train.describe()
columns_describe = df_train.describe().columns.tolist()
no_std_columns = []

for c in columns:
    std = df_desc[c]['std']

    #if unique_in_column/overall_length < 0.1 and c != 'label' and c != 'subject':
    if np.isnan(std):
        no_std_columns.append(c)

C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Tools\anaconda3\envs\stress01\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\T

In [27]:
no_std_columns

['bvp_BVP_entropy', 'acc_x_entropy', 'acc_y_entropy', 'acc_z_entropy']

In [28]:
columns_to_drop = list((set(constant_columns).union(set(no_std_columns))))

In [29]:
columns_to_drop

['eda_EDA_n_sign_changes',
 'temp_TEMP_peaks',
 'acc_y_entropy',
 'acc_l2_n_sign_changes',
 'acc_x_entropy',
 'acc_z_entropy',
 'temp_l2_n_sign_changes',
 'bvp_BVP_entropy',
 'temp_TEMP_n_sign_changes',
 'temp_l2_peaks',
 'eda_l2_n_sign_changes']

In [30]:
df_train = df_train.drop(columns=columns_to_drop)

## 4.2 Correlations

In [ ]:
plt.figure(figsize=(35, 35))
corr = df_train.corr(method='spearman')
heatmap = sns.heatmap(corr.sort_values(by='label', ascending=False),
                      vmin=-1, vmax=1, annot=True, fmt='.1g', cmap='BrBG')
heatmap.set_title('Features correlating with stress label', fontdict={'fontsize':15}, pad=16);

The image is really large, so we load a screenshot here
![alt text](eda-wesad-acc-bvp-eda-temp.jpg)

# 5. Documenting data lineage

The dataset contains
* BVP (blood volume pulse) sensor data in 64hz
* ACC: accelerometer data (x, y, z values) in 32hz
* EDA (electrodermal activity) in 4hz
* TEMP (temperature) in 4hz

The dataset is labeled (stress/no stress).

Script ```01-extract-data-from-wesad-dataset``` downloads the wesad study dataset. It unzips all included files. The raw data is stored in a pickle file provided from the researchers publishing the dataset. We load it from there, as well as the labels (stress/no stress). The results of the extraction are stored as a parquet file.

In script ```02-calculate-features```, we calculate features with the FLIRT library (https://flirt.readthedocs.io/en/latest/). In this notebook you're currently reading, we perform EDA. In the following steps, we might want to go back to feature calculation and calculate other/more features.

# 6. Observations from EDA

### Looking into the dataset
* Given the windows size and step size, we have 2765 rows and 264 features.
* We could also calculate feature via tsfresh and/or TSFEL; and we could try different parameters for window_length and window_step_size when using FLIRT.
* There are no missing values.
* We have 64% of negative cases (no stress) and 36% of positive cases (stress) in our data - we have to account for this when building and evaluating the model, e.g., by using appropriate evaluation metrics for imbalanced data.
* We do not have categorical variables, only numerical (count and float).

* There are a some columns that we drop, because they either have have the same value for each row, or it is impossible to calculate the standard deviation on them:
    * eda_EDA_n_sign_changes
    * temp_TEMP_peaks
    * acc_y_entropy
    * acc_l2_n_sign_changes
    * acc_x_entropy
    * acc_z_entropy
    * temp_l2_n_sign_changes
    * bvp_BVP_entropy
    * temp_TEMP_n_sign_changes
    * temp_l2_peaks
    * eda_l2_n_sign_changes
* The ranges of the values are quite far from each other - we should normalize/standardize.

### Correlations
* There are several correlated features. Because of the amount of features, we should apply an automated method for deciding which features to keep.